In [1]:
import pandas as pd
import json
import os.path

In [2]:
df = pd.read_csv('https://github.com/MrGeislinger/victorsothervector/raw/main/data/reading/all_reading-clean.csv')

# Clean data types 
df.duration = pd.to_timedelta(df.duration)
df.start = pd.to_datetime(df.start,)
df.end = pd.to_datetime(df.end,)

# df.to_csv('data/reading/all_reading-clean.csv', index=False)

In [3]:
def get_one_title_json(data: pd.DataFrame, title: str) -> dict:
    # Filter for specific title
    one_title = data[
        data.title == title
    ].copy()

    td_sum = one_title.duration.sum()
    td_components = td_sum.components
    td_total_seconds = td_sum.seconds

    # Build info json

    one_title_info = dict(
        title=one_title.title.iloc[0],
        start_time=one_title.start.min().strftime('%Y%m%d-%H:%M:%S'),
        end_time=one_title.end.max().strftime('%Y%m%d-%H:%M:%S'),
        total_seconds=td_total_seconds,
        duration_days=td_components.days,
        duration_hours=td_components.hours,
        duration_minutes=td_components.minutes,
        duration_seconds=td_components.seconds,
    )

    return one_title_info

In [4]:
template_header = '''
---
title: "*{book_title}*"
book:
  title: "{book_title}"
  title_simplified: {book_title_no_space}
reading:
  start: {start_time}
  end: {end_time}
  duration_total_seconds: {total_seconds}
  duration_hrs: {duration_hours}
  duration_mins: {duration_minutes}
  duration_secs: {duration_seconds}
  has_review: {has_review}
format:
  html:
    code-fold: true
jupyter: python3
---
'''

template = '''
## Reading Journey

* Started: {{{{< meta reading.start >}}}}
* Ended: {{{{< meta reading.end >}}}}
* Total Time Read: {{{{< meta reading.duration_hrs >}}}}hrs {{{{< meta reading.duration_mins >}}}}mins {{{{< meta reading.duration_secs >}}}}secs

```{{python}}
#| label: fig-reading-time
#| fig-cap: "Reading done for {book_title}"
%run _help_reading.py
import pandas as pd

df = pd.read_csv(
  'https://github.com/MrGeislinger/victorsothervector/raw/main/'
  'data/reading/all_reading-clean.csv'
)

book_name = """{book_title}"""
one_title = one_title_data(df, book_name)
one_title_summary = get_summary_by_day(one_title)
generate_plot(one_title_summary, book_name);
```
'''

template_with_review = '''
{{{{< include reviews/review-{book_title_no_space}.qmd >}}}}
'''

In [5]:
for t in df.title.unique():
    print(t)
    one_title = get_one_title_json(
        data=df,
        title=t,
    )
    t_ = (
        t
        .replace('\'','')
        .replace(' ','_')
        .lower()
        .replace(':','–')
    )
    # Check if a review exists
    file_path = f'books/reviews/review-{t_}.qmd'
    has_review = os.path.exists(file_path)
    # Template changes if review
    if has_review:
        final_template = template_header + template_with_review + template
    else:
        final_template = template_header + template

    with open(f'books/book-{t_}.qmd', 'w') as f_qmd:
        f_qmd.write(
            final_template.format(
                book_title=t,
                book_title_no_space=t_,
                start_time=one_title.get('start_time'),
                end_time=one_title.get('end_time'),
                has_review=has_review,
                total_seconds=one_title.get('total_seconds'),
                duration_hours=one_title.get('duration_hours'),
                duration_minutes=one_title.get('duration_minutes'),
                duration_seconds=one_title.get('duration_seconds'),
            )
        )
    # with open(f'data/reading/{t_}.json', 'w') as f:
    #     json.dump(one_title, f)
    print('\t done')

Hyperion
	 done
Reinforcement Learning
	 done
Yellowface
	 done
Dune
	 done
Turtles All the Way Down
	 done
Acoustic and Auditory Phonetics
	 done
In the Land of Invented Languages
	 done
Through the Language Glass
	 done
The Infinite Gift
	 done
Things to Make and Do in the Fourth Dimension
	 done
Disability Visibility
	 done
Infectious Generosity
	 done
How to Speak Whale
	 done
Death's End
	 done
Talking Hands
	 done
Humble Pi
	 done
High Performance Computing
	 done
Swearing is Good for You
	 done
The Dark Forest
	 done
The Precipice
	 done
The Little Book of Deep Learning
	 done
Practical Fairness
	 done
The Three-Body Problem
	 done
Weapons of Math Destruction
	 done
How Google Works
	 done
Python Data Science Handbook
	 done
Algorithms to Live By
	 done
Software Engineering at Google
	 done
Building an Anonymization Pipeline
	 done
Generative Deep Learning, 2nd Ed
	 done
What If
	 done
Hello World: Being Human in the Age of Algorithms
	 done
Other Minds
	 done
Generative Deep Le